# Running Gromacs Simulation Chains on a Compute Cluster

This tutorial introduces three high-level concepts that are meant to make your life easier:
1. Command chains
2. Command chain generators
3. Coffe cluster jobs




In [ ]:
%reload_ext autoreload

In [ ]:
%autoreload 2

## Command Chains 

In many situations, simulations are not executed individually, but in a larger context.
For example, a typical MD simulation requires energy minimization and equilibration runs. These calculations have to be run prior to the actual production run.

In this tutorial, we will use a typical chain of individual simulation tasks (the mdp files are found in the present directory).

|name | Task | mdp file | 
|-----|-----|-----------|
|emin1 | Energy Minimization (Steepest Descent Method)|em_steep.mdp|
|emin2 | Energy Minimization (L-BFGS Method) | em_l-bfgs.mdp
|nvt | NVT Equilibration |nvt.mdp|
|npt | NPT Equilibration |npt.mdp|
|prod | Production Run |md.mdp|


In order to facilitate such simulation chains, COFFE allows simulations to be tied together in command chains.
A command chain (as implemented in *coffe.core.cmdchain*) is basically a list of calculations, where each calculation is represented by an instance of a callable class, cf. Tutorial 02.

The CommandChain itself has a call function that just calls its items one after the other.




## Command Chain Generators
Even simulation chains are often embedded in a broader context. A few typical examples are:
- observables are often evaluated at different temperatures or pressures,
- free energy perturbation uses multiple simulations, e.g. to alchemically grow a molecule in a box of solvent,
- force field parameterization usually requires simulations for different topology files,
- many simulation studies require simulations of different systems.

Typically, these different simulations share most settings.
In order to facilitate setting up many simulations, COFFE implements a high-level command chain generator for coffe simulations.

A generator is set up with the shared settings (most importantly mdp files of the individual simulation tasks). 


In [ ]:
from coffe.gmx import sim, simgen
generator = simgen.GmxChainGenerator(
    names=["emin1", "emin2", "nvt", "npt", "prod"], 
    mdp_files=["em_steep.mdp","em_l-bfgs.mdp", "nvt.mdp", "npt.mdp", "md.mdp"], 
    mdp_options=None # can be used to overwrite options in the mdp files
)

*(Of course, it could also be constructed via a configuration file.)*

This generator can now be used to automatically generate simulation chains.
Using two boxes (hexadecane and octane),


In [ ]:
from coffe.gmx import boxes
hexa_gro, hexa_top = boxes.gmx_mkbox_homogeneous(cfg_file="boxes.cfg", section="hexa_126")
octa_gro, octa_top = boxes.gmx_mkbox_homogeneous(cfg_file="boxes.cfg", section="octa_256")

we can now generate simulation chains for each of these systems:

In [ ]:
hexa_chain_T298 = generator.generate("output_hexa", hexa_gro, hexa_top)
octa_chain_T298 = generator.generate("output_octa", octa_gro, octa_top)

We could now run these simulation chains locally by calling

```
hexa_chain_T298()
octa_chain_T298()
```

However, this can take quite long.
Usually, we want to run simulations on a cluster.
To facilitate cluster submission, coffe has a class *ClusterJob*.


## Coffe Cluster Jobs

Cluster jobs provide an interface for sumitting commands/simulation instances to a cluster. A cluster job is created as follows: 

In [ ]:
from coffe.core import cluster

queueing=None
batch_template=None
job_name="coffe_job"
work_dir="./output_cluster"
job = cluster.ClusterJob(queueing, batch_template, job_name, work_dir)


On an actual compute cluster, you will need to set queueing to "torque" or "slurm", depending on which submission system your cluster uses. Moreover, you need to provide a batch template file, i.e. a file that contains only the preamble of a batch script. (By setting both options to ```None```, the job will still run locally.) The commands will be inserted automatically by adding simulation classes to the cluster job:

In [ ]:
job += hexa_chain_T298

Each simulation instance that is added to a cluster must have an empty ```__call__``` function.
It is also possible to append strings, which will be interpreted as shell commands.
This can be helpful, e.g., to run analyses after the actual simulation.

In [ ]:
# something trivial
job += 'echo "Hallo" '

To start a job, just call

In [ ]:
# job_id = job.submit()    

```job.submit()``` returns a job id, to track your job.

Moreover, it provides functionality to check the status of a job or kill a job.
```
job.get_status() 
job.kill()
```

job.get_status() returns one of the following strings:

|result | meaning |
|--|---|
|not written | batch script is not written, yet |
|not submitted | job is not submitted, yet|
|queueing | job is in queue |
|running | job is running |
|completed | job is completed  |
|error | job failed|




## Putting It All Together
In the above code, the actual submission was commented out, to prevent this notebook from starting expensive calculations.
To test the whole workflow on a real cluster, checkout the subdirectory "workflow". This subdirectory contains an executable python script "workflow.py" and a configuration file "workflow.cfg" to run the discussed simulations on a real cluster.

Note that you may need to adapt the batch script and queueing system to your cluster configuration.